# 📊 Proyecto Avanzado: Análisis de Ventas con Pandas + Matplotlib (PLUS)
Este cuaderno guía un análisis **end-to-end**: carga/validación de datos, EDA, limpieza, *feature engineering*, KPIs,
visualizaciones y analítica de clientes (RFM y cohortes). Incluye **tareas** con **soluciones ocultas**.

**Objetivos:**
- Practicar carga y validación de datos con `pandas`.
- Crear KPIs de negocio (ingresos, ticket medio, devoluciones, top productos/clientes).
- Usar `groupby`, *pivot tables*, re-muestreo por mes y gráficos con **Matplotlib**.
- Analizar clientes con **RFM** y **cohortes**.

**Entregables:** CSV con resultados clave, 3 gráficos y un breve texto con hallazgos.

**ACLARACIONES:**
EDA (Análisis Exploratorio de Datos) te dice qué hay y cómo están los datos; RFM (Recencia -días de la última compra -, Frecuencia y Valor Monetario) te dice qué clientes son más valiosos/activos; cohortes (Grupo de clientes definido por su momento de alta) te dice cómo evoluciona la retención en el tiempo.

## 0) Preparación: librerías y helpers

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from datetime import datetime
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 120)

In [ ]:
# Importamos pandas para trabajar con tablas (tipo Excel)
import pandas as pd

# Importamos numpy para cálculos numéricos
import numpy as np

# Path sirve para manejar rutas de archivos y carpetas
from pathlib import Path

# matplotlib.pyplot permite hacer gráficas
import matplotlib.pyplot as plt

# datetime permite trabajar con fechas y horas
from datetime import datetime

# Configuramos pandas para mostrar máximo 10 filas en pantalla
pd.set_option('display.max_rows', 10)

# Configuramos pandas para que las tablas usen hasta 120 caracteres de ancho
pd.set_option('display.width', 120)

In [ ]:
📋 Resumen corto

Este código prepara el entorno:

Carga librerías para datos (pandas, numpy), archivos (Path), gráficas (matplotlib) y fechas (datetime).

Configura pandas para que muestre las tablas de forma más ordenada.

## 1) Dataset: generar o cargar
Si existe `ventas_simuladas.csv`, lo cargamos. Si no, generamos uno reproducible.

In [ ]:
def generar_dataset_csv(ruta='ventas_simuladas.csv', seed=42):
    rng = np.random.default_rng(seed)
    dates = pd.date_range('2023-01-01', '2024-12-31', freq='D')
    products = [
        ('Teclado','Periféricos'),('Ratón','Periféricos'),('Monitor','Pantallas'),
        ('Portátil','Ordenadores'),('Disco SSD','Almacenamiento'),('Impresora','Periféricos'),
        ('Auriculares','Periféricos'),('Base refrigeradora','Periféricos'),('Webcam','Periféricos')
    ]
    rows = []
    order_id = 1
    for d in dates:
        for _ in range(rng.integers(5, 20)):
            prod, cat = products[rng.integers(0, len(products))]
            units = int(rng.integers(1, 5))
            base = 50 if cat=='Periféricos' else (300 if cat in ['Ordenadores','Pantallas'] else 100)
            price = float(rng.normal(base, base*0.35))
            price = max(price, 5)
            returned = rng.random() < 0.05
            rows.append((order_id, d, rng.integers(1, 801), prod, cat, units, round(price,2), returned))
            order_id += 1
    df = pd.DataFrame(rows, columns=['order_id','order_date','customer_id','product','category','units','unit_price','returned'])
    df.to_csv(ruta, index=False)
    return ruta

csv_path = Path('ventas_simuladas.csv')
if not csv_path.exists():
    generar_dataset_csv(csv_path)
df = pd.read_csv(csv_path, parse_dates=['order_date'])
df.head()

In [ ]:
# Función que genera un dataset de ventas simuladas y lo guarda en un CSV
def generar_dataset_csv(ruta='ventas_simuladas.csv', seed=42):
    rng = np.random.default_rng(seed)  # Generador aleatorio con semilla fija
    dates = pd.date_range('2023-01-01', '2024-12-31', freq='D')  # Todas las fechas entre 2023 y 2024
    
    # Lista de productos con su categoría
    products = [
        ('Teclado','Periféricos'),('Ratón','Periféricos'),('Monitor','Pantallas'),
        ('Portátil','Ordenadores'),('Disco SSD','Almacenamiento'),('Impresora','Periféricos'),
        ('Auriculares','Periféricos'),('Base refrigeradora','Periféricos'),('Webcam','Periféricos')
    ]
    
    rows = []        # Aquí guardaremos todas las ventas
    order_id = 1     # ID inicial de pedidos
    
    # Para cada día en el rango de fechas
    for d in dates:
        # Simulamos entre 5 y 20 ventas diarias
        for _ in range(rng.integers(5, 20)):
            # Elegir producto y categoría aleatoriamente
            prod, cat = products[rng.integers(0, len(products))]
            units = int(rng.integers(1, 5))  # Unidades compradas (1 a 4)
            
            # Precio base según la categoría
            base = 50 if cat=='Periféricos' else (300 if cat in ['Ordenadores','Pantallas'] else 100)
            price = float(rng.normal(base, base*0.35))  # Precio aleatorio alrededor del base
            price = max(price, 5)  # Nunca menor a 5
            
            # 5% de probabilidad de devolución
            returned = rng.random() < 0.05
            
            # Guardar esta venta
            rows.append((order_id, d, rng.integers(1, 801), prod, cat, units, round(price,2), returned))
            order_id += 1  # Siguiente ID
    
    # Convertir a DataFrame y guardar en CSV
    df = pd.DataFrame(rows, columns=['order_id','order_date','customer_id','product','category','units','unit_price','returned'])
    df.to_csv(ruta, index=False)
    return ruta

# Ruta donde se guarda el CSV
csv_path = Path('ventas_simuladas.csv')

# Si no existe el archivo, lo creamos
if not csv_path.exists():
    generar_dataset_csv(csv_path)

# Cargar el dataset en un DataFrame
df = pd.read_csv(csv_path, parse_dates=['order_date'])

# Mostrar las primeras filas
df.head()


In [ ]:
📋 Resumen corto

Este código:

Genera un archivo CSV con ventas ficticias de distintos productos (si no existe).

Cada venta tiene: id, fecha, cliente, producto, categoría, unidades, precio y si fue devuelto.

Luego lee ese archivo y muestra las primeras filas.

## 2) Validación y limpieza mínima

In [ ]:
# Tipos y nulos
display(df.info())
nulos = df.isna().sum()
nulos

In [ ]:
# Ver un resumen del DataFrame: número de filas, columnas y tipos de datos
display(df.info())

# Contar cuántos valores nulos (vacíos) hay en cada columna
nulos = df.isna().sum()

# Mostrar el conteo de nulos
nulos

In [ ]:
📋 Resumen corto

Este bloque sirve para verificar la calidad de los datos:

df.info() → revisa la estructura y tipos de las columnas.

df.isna().sum() → revisa si hay valores faltantes en alguna columna.

En este dataset, lo normal es que no haya nulos (0 en todas las columnas).

In [ ]:
# Reglas de negocio simples
assert (df['units']>=1).all(), 'Units debe ser >= 1'
assert (df['unit_price']>0).all(), 'unit_price debe ser > 0'
df['returned'] = df['returned'].astype(bool)
df['category'] = df['category'].astype('category')
df['product'] = df['product'].astype('category')
df['order_date'] = pd.to_datetime(df['order_date'])
df['revenue'] = df['units'] * df['unit_price']
df.head()

In [ ]:
# Reglas de negocio: validar que no haya datos inválidos
assert (df['units']>=1).all(), 'Units debe ser >= 1'        # Todas las ventas deben tener al menos 1 unidad
assert (df['unit_price']>0).all(), 'unit_price debe ser > 0'  # El precio debe ser siempre positivo

# Convertir columnas a tipos más adecuados
df['returned'] = df['returned'].astype(bool)          # True/False
df['category'] = df['category'].astype('category')    # Categorías de productos
df['product'] = df['product'].astype('category')      # Productos

# Asegurar que las fechas son del tipo datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Crear una nueva columna: ingresos = unidades * precio unitario
df['revenue'] = df['units'] * df['unit_price']

# Mostrar las primeras filas con la nueva columna
df.head()

In [ ]:
📋 Resumen corto

Este bloque:

Valida que no haya datos incorrectos (unidades < 1, precios ≤ 0).

Convierte columnas a tipos adecuados (bool, category, datetime).

Calcula una columna nueva: revenue (ingresos por venta).

## 3) EDA rápida

In [ ]:
df.describe(include='all')

In [ ]:
# Generar un resumen estadístico de todas las columnas
# - Números → media, desviación, min, max, percentiles
# - Texto/categorías → cuántos valores únicos, el más frecuente
# - Booleanos → valor más frecuente (True/False)
# - Fechas → muestra count, unique, top, freq
df.describe(include='all')

In [ ]:
📋 Resumen corto

df.describe(include='all') → genera un informe general de todo el dataset:

Estadísticas de números (promedio, mínimos, máximos).

Estadísticas de categorías (valor más común, número de categorías).

Estadísticas de booleanos y fechas.

In [ ]:
df.groupby('category')['revenue'].sum().sort_values(ascending=False).head(10)

In [ ]:
# Calcular los ingresos totales agrupados por categoría de producto
df.groupby('category')['revenue'].sum() \
    .sort_values(ascending=False) \  # Ordenar de mayor a menor
    .head(10)  # Mostrar solo los 10 primeros

In [ ]:
📋 Resumen corto

Este comando calcula:
👉 ¿Qué categorías de productos generan más dinero en total?
Es un análisis clave para la estrategia de negocio.

### Tarea 1: ¿Cuál es el rango de fechas y el nº de pedidos únicos?

<details><summary><b>Ver solución</b></summary>

```python
df['order_date'].min(), df['order_date'].max(), df['order_id'].nunique()
```
</details>

## 4) KPIs básicos

In [ ]:
# KPI 1: Ingresos por mes
df['month'] = df['order_date'].dt.to_period('M').dt.to_timestamp()
ingresos_mes = df.groupby('month')['revenue'].sum().reset_index()
ingresos_mes.head()

In [ ]:
# Crear columna "month" que representa el mes de cada venta
df['month'] = df['order_date'].dt.to_period('M').dt.to_timestamp()

# Calcular ingresos totales por mes
ingresos_mes = df.groupby('month')['revenue'].sum().reset_index()

# Mostrar los primeros 5 meses con sus ingresos
ingresos_mes.head()

In [ ]:
📋 Resumen corto

Este bloque:

Añade una columna month con el mes de cada venta.

Agrupa todas las ventas por mes.

Calcula los ingresos totales por mes.

👉 Resultado: una tabla con los ingresos de la empresa mes a mes.

In [ ]:
# KPI 2: Ticket medio por pedido
ticket_medio = df.groupby('order_id')['revenue'].sum().mean()
ticket_medio

In [ ]:
# Calcular ticket medio (promedio de ingresos por pedido)
# 1. Agrupar ventas por pedido (order_id)
# 2. Sumar ingresos de cada pedido
# 3. Calcular el promedio de todos los pedidos
ticket_medio = df.groupby('order_id')['revenue'].sum().mean()

# Mostrar el resultado
ticket_medio

In [ ]:
📋 Resumen corto

Este bloque calcula el ticket medio:
👉 El promedio de ingresos por pedido (order_id).

Es un KPI fundamental porque te dice:

Si el ticket medio sube → los clientes compran más cosas por pedido (o más caras).

Si baja → tal vez compran menos productos o más baratos.

In [ ]:
# KPI 3: Top 10 productos por ingresos
top_prod = df.groupby('product')['revenue'].sum().sort_values(ascending=False).head(10)
top_prod

In [ ]:
# KPI 3: Top 10 productos por ingresos
# 1. Agrupar ventas por producto
# 2. Sumar ingresos de cada producto
# 3. Ordenar de mayor a menor
# 4. Tomar los 10 primeros
top_prod = df.groupby('product')['revenue'].sum().sort_values(ascending=False).head(10)

# Mostrar los productos más rentables
top_prod

In [ ]:
📋 Resumen corto

Este bloque calcula el top 10 de productos por ingresos totales.
👉 Es un KPI importante para identificar qué productos son más rentables y estratégicos.

In [ ]:
# KPI 4: Tasa de devolución
tasa_dev = df['returned'].mean()
tasa_dev

In [ ]:
# KPI 4: Tasa de devolución
# La columna 'returned' es booleana (True=devuelto, False=no devuelto)
# La media nos da el porcentaje de productos devueltos
tasa_dev = df['returned'].mean()

# Mostrar la tasa de devolución (entre 0 y 1)
tasa_dev

In [ ]:
📋 Resumen corto

Este bloque calcula qué proporción de productos se devuelven.

KPI importante para medir satisfacción del cliente y problemas de calidad.

## 5) Visualizaciones con Matplotlib (una figura por gráfico)

In [ ]:
# Ingresos por mes (línea)
ingresos_mes.plot(x='month', y='revenue', kind='line', title='Ingresos por mes')
plt.show()

In [ ]:
# Graficar ingresos totales por mes
ingresos_mes.plot(
    x='month',        # Eje X = meses
    y='revenue',      # Eje Y = ingresos
    kind='line',      # Tipo de gráfico = línea
    title='Ingresos por mes'  # Título del gráfico
)

# Mostrar el gráfico
plt.show()

In [ ]:
📋 Resumen corto

Este bloque dibuja un gráfico de línea que muestra:

La evolución de los ingresos mes a mes.

Permite detectar temporadas altas y bajas, tendencias, crecimientos o caídas en ventas.

In [ ]:
# Ingresos por categoría (barra)
df.groupby('category')['revenue'].sum().sort_values().plot(kind='bar', title='Ingresos por categoría')
plt.show()

In [ ]:
# Graficar ingresos totales por categoría de producto
df.groupby('category')['revenue'].sum() \   # Sumar ingresos por categoría
    .sort_values() \                        # Ordenar de menor a mayor
    .plot(kind='bar', title='Ingresos por categoría')  # Graficar como barras

# Mostrar el gráfico
plt.show()

In [ ]:
📋 Resumen corto

Este bloque genera un gráfico de barras que permite:

Comparar rápidamente qué categorías generan más ingresos.

Visualizar fácilmente diferencias entre categorías.

In [ ]:
# Top 10 productos (barra)
df.groupby('product')['revenue'].sum().sort_values(ascending=False).head(10).plot(kind='bar', title='Top 10 productos')
plt.show()

In [ ]:
# Graficar los 10 productos que más ingresos generan
df.groupby('product')['revenue'].sum() \        # Sumar ingresos por producto
    .sort_values(ascending=False) \            # Ordenar de mayor a menor
    .head(10) \                                # Tomar los 10 primeros
    .plot(kind='bar', title='Top 10 productos') # Graficar como barras

# Mostrar el gráfico
plt.show()

In [ ]:
📋 Resumen corto

Este gráfico permite ver rápidamente:

Qué productos generan más ingresos.

Identificar productos estratégicos o estrellas del catálogo.

### Tarea 2: Representa la evolución de la **tasa de devolución** por mes

<details><summary><b>Ver solución</b></summary>

```python
dev_mes = df.groupby('month')['returned'].mean().reset_index()
dev_mes.plot(x='month', y='returned', kind='line', title='Tasa de devolución por mes')
plt.show()
```
</details>

## 6) Analítica de clientes: **RFM**

In [ ]:
# Calcula Recency (días desde última compra), Frequency (nº pedidos), Monetary (ingresos)
ref_date = df['order_date'].max() + pd.Timedelta(days=1)
agg = df.groupby('customer_id').agg(
    last_purchase=('order_date','max'),
    frequency=('order_id','nunique'),
    monetary=('revenue','sum')
).reset_index()
agg['recency'] = (ref_date - agg['last_purchase']).dt.days
agg.head()

In [ ]:
# Fecha de referencia para calcular recency (un día después de la última venta)
ref_date = df['order_date'].max() + pd.Timedelta(days=1)

# Agregar métricas por cliente
agg = df.groupby('customer_id').agg(
    last_purchase=('order_date','max'),  # Última compra
    frequency=('order_id','nunique'),    # Número de pedidos
    monetary=('revenue','sum')           # Total gastado
).reset_index()

# Calcular Recency = días desde la última compra
agg['recency'] = (ref_date - agg['last_purchase']).dt.days

# Mostrar las primeras filas
agg.head()

In [ ]:
📋 Resumen corto

Este bloque crea un resumen por cliente para análisis RFM:

    Recency (R) → días desde la última compra

    Frequency (F) → número de pedidos

    Monetary (M) → ingresos totales del cliente

Estas métricas son clave para:

    Identificar clientes leales (alta frecuencia, alto gasto)

    Detectar clientes inactivos (recency alto)

    Planificar campañas de marketing o retención.

In [ ]:
# Discretización por quintiles para r, f, m → scores 1–5
def quintil_scorer(s, reverse=False):
    q = s.rank(pct=True)
    sc = (q*5).apply(np.ceil).astype(int)
    sc = sc.clip(1,5)
    return 6 - sc if reverse else sc
agg['R'] = quintil_scorer(agg['recency'], reverse=True)  # menor recency → mejor
agg['F'] = quintil_scorer(agg['frequency'])
agg['M'] = quintil_scorer(agg['monetary'])
agg['RFM'] = agg['R'].astype(str)+agg['F'].astype(str)+agg['M'].astype(str)
agg.sort_values('monetary', ascending=False).head(10)

In [ ]:
# Función para asignar scores de 1 a 5 según quintiles
def quintil_scorer(s, reverse=False):
    q = s.rank(pct=True)                # percentil de cada valor
    sc = (q*5).apply(np.ceil).astype(int)  # asignar quintil 1-5
    sc = sc.clip(1,5)                   # asegurar rango 1-5
    return 6 - sc if reverse else sc    # invertir si reverse=True

# Asignar scores RFM
agg['R'] = quintil_scorer(agg['recency'], reverse=True)  # menor recency = mejor
agg['F'] = quintil_scorer(agg['frequency'])
agg['M'] = quintil_scorer(agg['monetary'])

# Crear columna combinada RFM
agg['RFM'] = agg['R'].astype(str) + agg['F'].astype(str) + agg['M'].astype(str)

# Mostrar top 10 clientes por gasto
agg.sort_values('monetary', ascending=False).head(10)

In [ ]:
📋 Resumen corto

Este bloque completa el análisis RFM:

    Convierte recency, frequency y monetary en scores 1–5 según quintiles.
    Combina los scores en una cadena RFM para segmentar clientes.
    Permite identificar clientes valiosos y priorizar campañas.

### Tarea 3: ¿Qué porcentaje de ingresos aportan los **top 10%** de clientes por `monetary`?

<details><summary><b>Ver solución</b></summary>

```python
n = max(1, int(len(agg)*0.10))
top = agg.sort_values('monetary', ascending=False).head(n)
pct = top['monetary'].sum() / agg['monetary'].sum()
pct
```
</details>

## 7) Analítica de **cohortes** (retención mensual)
Definimos la cohorte por **mes de primera compra** y construimos una tabla de retención.

In [ ]:
df_small = df[['customer_id','order_id','order_date','revenue']].copy()
df_small['first_month'] = df_small.groupby('customer_id')['order_date'].transform('min').dt.to_period('M').dt.to_timestamp()
df_small['month'] = df_small['order_date'].dt.to_period('M').dt.to_timestamp()
df_small['cohort_index'] = ((df_small['month'].dt.year - df_small['first_month'].dt.year)*12 +
                             (df_small['month'].dt.month - df_small['first_month'].dt.month))
retencion = (df_small
    .groupby(['first_month','cohort_index'])['customer_id']
    .nunique()
    .reset_index())
base = retencion[retencion['cohort_index']==0][['first_month','customer_id']].rename(columns={'customer_id':'n0'})
tabla = retencion.merge(base, on='first_month')
tabla['ret'] = tabla['customer_id']/tabla['n0']
cohort_pivot = tabla.pivot(index='first_month', columns='cohort_index', values='ret').fillna(0)
cohort_pivot.head()

In [ ]:
# Selección de columnas relevantes
df_small = df[['customer_id','order_id','order_date','revenue']].copy()

# Primer mes de compra por cliente (cohorte)
df_small['first_month'] = df_small.groupby('customer_id')['order_date'].transform('min').dt.to_period('M').dt.to_timestamp()

# Mes de cada compra
df_small['month'] = df_small['order_date'].dt.to_period('M').dt.to_timestamp()

# Índice de cohorte: meses desde la primera compra
df_small['cohort_index'] = ((df_small['month'].dt.year - df_small['first_month'].dt.year)*12 +
                             (df_small['month'].dt.month - df_small['first_month'].dt.month))

# Contar clientes únicos por cohorte y mes relativo
retencion = (df_small
    .groupby(['first_month','cohort_index'])['customer_id']
    .nunique()
    .reset_index())

# Base de clientes iniciales (mes 0)
base = retencion[retencion['cohort_index']==0][['first_month','customer_id']].rename(columns={'customer_id':'n0'})

# Calcular tasa de retención
tabla = retencion.merge(base, on='first_month')
tabla['ret'] = tabla['customer_id']/tabla['n0']

# Pivot para crear tabla de cohortes
cohort_pivot = tabla.pivot(index='first_month', columns='cohort_index', values='ret').fillna(0)

# Mostrar primeras filas
cohort_pivot.head()

In [ ]:
📋 Resumen corto

Este bloque calcula la retención de clientes por cohorte mensual:

    Define la cohorte de cada cliente según su primer mes de compra.
    Calcula la diferencia de meses entre cada compra y la primera compra → cohort_index.
    Cuenta cuántos clientes siguen comprando mes a mes.
    Crea una tabla donde se ve claramente cómo disminuye la retención con el tiempo.

In [ ]:
# Visualización rápida de cohortes (matriz)
plt.figure()
plt.imshow(cohort_pivot.values, aspect='auto')
plt.title('Retención por cohortes (proporción)')
plt.xlabel('Meses desde primera compra')
plt.ylabel('Cohorte (mes inicio)')
plt.colorbar()
plt.show()

In [ ]:
plt.figure()  # Crear nueva figura

# Dibujar matriz de retención
plt.imshow(cohort_pivot.values, aspect='auto')

# Etiquetas y título
plt.title('Retención por cohortes (proporción)')
plt.xlabel('Meses desde primera compra')
plt.ylabel('Cohorte (mes inicio)')

# Barra de colores
plt.colorbar()

# Mostrar gráfico
plt.show()

In [ ]:
📋 Resumen corto

Este bloque crea un heatmap básico de retención por cohortes:

    Filas → cada cohorte mensual
    Columnas → meses desde la primera compra
    Color → proporción de clientes que regresan
    Permite visualizar rápidamente la fidelidad y la caída de clientes con el tiempo.

### Tarea 4: Calcular el **LTV** aproximado por cohorte como suma de ingresos medios por cliente a 3 meses

<details><summary><b>Ver solución (idea)</b></summary>

```python
ingresos_cliente_mes = (df_small
    .groupby(['first_month','month'])
    .agg(ingresos=('revenue','sum'), clientes=('customer_id','nunique'))
    .reset_index())
ingresos_cliente_mes['cohort_index'] = ((ingresos_cliente_mes['month'].dt.year - ingresos_cliente_mes['first_month'].dt.year)*12 +
                                        (ingresos_cliente_mes['month'].dt.month - ingresos_cliente_mes['first_month'].dt.month))
ing_cli = ingresos_cliente_mes.pivot_table(index='first_month', columns='cohort_index', values='ingresos')
cli = ingresos_cliente_mes.pivot_table(index='first_month', columns='cohort_index', values='clientes')
arpu = ing_cli/cli
ltv3 = arpu[[0,1,2]].sum(axis=1)
ltv3.head()
```
</details>

## 8) Export de resultados

In [ ]:
ingresos_mes.to_csv('kpi_ingresos_mensuales.csv', index=False)
agg[['customer_id','R','F','M','RFM','recency','frequency','monetary']].to_csv('rfm_clientes.csv', index=False)
cohort_pivot.to_csv('retencion_cohortes.csv')
print('✅ Exportados: kpi_ingresos_mensuales.csv, rfm_clientes.csv, retencion_cohortes.csv')

In [ ]:
# Exportar ingresos mensuales a CSV
ingresos_mes.to_csv('kpi_ingresos_mensuales.csv', index=False)

# Exportar métricas RFM de clientes a CSV
agg[['customer_id','R','F','M','RFM','recency','frequency','monetary']].to_csv('rfm_clientes.csv', index=False)

# Exportar tabla de retención por cohortes a CSV
cohort_pivot.to_csv('retencion_cohortes.csv')

# Mensaje de confirmación
print('✅ Exportados: kpi_ingresos_mensuales.csv, rfm_clientes.csv, retencion_cohortes.csv')

In [ ]:
📋 Resumen corto

Este bloque permite guardar todos los resultados de los KPIs y análisis RFM/cohortes en archivos CSV, para:

    Usarlos en herramientas de reporting.
    Compartir con otros miembros del equipo.
    Tener un backup de los análisis realizados.

## 9) Retos extra (para nota)
1) **Anomalías**: detecta días con ingresos > μ+3σ y coméntalos.
2) **Segmentación**: clasifica clientes en 4 grupos (p.ej. KMeans sobre (recency, frequency, monetary)).
3) **AB testing simulado**: crea una bandera de campaña y compara ticket medio con test estadístico.
4) **Performance**: compara tiempo de `groupby` tras convertir `product` y `category` a `category`.

#### <span style="background-color: cyan"> 1. Anomalías: detecta días con ingresos > μ+3σ y coméntalos.

In [ ]:
# Agrupar ingresos por día
ingresos_dia = df.groupby('order_date')['revenue'].sum()

# Calcular media y desviación
mu = ingresos_dia.mean()
sigma = ingresos_dia.std()

# Detectar días con ingresos > μ + 3σ
anomalos = ingresos_dia[ingresos_dia > mu + 3*sigma]
anomalos

Explicación:

    - μ + 3σ es un criterio típico para detectar outliers (días con ingresos muy superiores al promedio).
    - anomalos devuelve los días con ingresos inusualmente altos.
    - Comentario: podrías inspeccionar esos días para ver si hubo ofertas especiales, Black Friday o errores en los datos.

#### <span style="background-color: cyan">  2. Segmentación: clasifica clientes en 4 grupos (p.ej. KMeans sobre (recency, frequency, monetary)).

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Variables RFM
X = agg[['recency','frequency','monetary']]

# Escalar para KMeans
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# KMeans con 4 clusters
kmeans = KMeans(n_clusters=4, random_state=42)
agg['cluster'] = kmeans.fit_predict(X_scaled)

# Resumen de clusters
agg.groupby('cluster')[['recency','frequency','monetary']].mean()

Explicación:

- Escalamos porque recency, frequency y monetary tienen diferentes magnitudes.
- KMeans agrupa clientes en 4 segmentos, por ejemplo:
    - Cluster 0 → VIP
    - Cluster 1 → Dormidos
    - Cluster 2 → Nuevos
    - Cluster 3 → Regulares
- Luego puedes analizar el perfil promedio de cada cluster.

#### <span style="background-color: cyan">  3. AB testing simulado: crea una bandera de campaña y compara ticket medio con test estadístico.

In [ ]:
from scipy.stats import ttest_ind

# Crear bandera aleatoria de campaña
np.random.seed(42)
df['campaign'] = np.random.choice([0,1], size=len(df))

# Ticket medio por grupo
ticket_camp = df.groupby('campaign').apply(lambda x: (x['units']*x['unit_price']).mean())
ticket_camp

# Test t para comparar grupos
grupo0 = df[df['campaign']==0]['units']*df[df['campaign']==0]['unit_price']
grupo1 = df[df['campaign']==1]['units']*df[df['campaign']==1]['unit_price']
t_stat, p_val = ttest_ind(grupo0, grupo1)
t_stat, p_val

Explicación:

    - campaign simula un grupo de control (0) y test (1).
    - Calculamos ticket medio por grupo y usamos un t-test para ver si la diferencia es significativa.
    - Si p_val < 0.05 → diferencia estadísticamente significativa.

#### <span style="background-color: cyan">  4. Performance: compara tiempo de groupby tras convertir product y category a category.

In [ ]:
import time

# Copiar df
df_perf = df.copy()

# Medir tiempo antes de convertir a category
start = time.time()
df_perf.groupby('product')['revenue'].sum()
time_raw = time.time() - start

# Convertir columnas a category
df_perf['product'] = df_perf['product'].astype('category')
df_perf['category'] = df_perf['category'].astype('category')

# Medir tiempo después de convertir
start = time.time()
df_perf.groupby('product')['revenue'].sum()
time_cat = time.time() - start

time_raw, time_cat

In [ ]:
Explicación:

    - Convertir columnas de texto repetitivas a category reduce memoria y acelera operaciones.
    - time_raw → tiempo antes
    - time_cat → tiempo después
    - Compara ambos para comentar la mejora de performance.

## 10) Rúbrica (10 puntos)
- Carga/validación y limpieza básica (2)
- KPIs correctos y bien interpretados (2)
- Gráficos claros (3) con conclusiones (2)
- Analítica de clientes (RFM/cohortes) (3)
- Export de resultados (1)
- Calidad del cuaderno (narrativa, orden, reproducibilidad) (+1 bonus)